In [6]:
import cvxpy as cp
import numpy as np
import pandas as pd

# 데이터 초기화
months = 2
# month[0]: 첫째 달, month[1]: 둘째 달

factories = 3
# factories[0]: Chandler, factories[1]: Glendale, factories[2]: Tucson

productions = 9
# productions[0]: Steel shafts, productions[1]: Graphite shafts, productions[2]: Forged iron heads, productions[3]: Metal wood heads, productions[4]: Titanium insert head
# productions[5]: Set(steel, iron, metal), productions[6]: Set(steel, iron, insert), productions[7]: Set(graphite, iron, metal), productions[8]: set(graphite, iron, insert)


# 엑셀로부터 데이터 불러오기
# 가격 초기화
price = pd.read_excel('OR.xlsx', sheet_name = 'Revenue').values.transpose()

# 생산 비용 초기화
cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_2').values.transpose()), axis = 0)

# 재고 비용 초기화
inventory_cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_2').values.transpose()), axis = 0)

# 광고 비용 초기화
advertise_cost = pd.read_excel('OR.xlsx', sheet_name = 'Advertisement Cost').values.transpose()

# 노동 시간 초기화
labor_time = pd.read_excel('OR.xlsx', sheet_name = 'Labor').values.transpose()

labor_limit = np.array([12000, 15000, 22000])

# 포장 시간 초기화
packing_time = pd.read_excel('OR.xlsx', sheet_name = 'Packing').values.transpose()

packing_limit = np.array([20000, 40000, 35000])

# 조립 시간 초기화
assembly_time = pd.read_excel('OR.xlsx', sheet_name = 'Assembly_Time').values.transpose()

assembly_limit = np.array([5500, 5000, 6000])

# 수요 조건 초기화
demand_bound = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Demand Lower bound').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Demand Upper bound').values.transpose()), axis = 0)

# 변수 초기화
sold = {}
manufacture = {}
inventory = {}

# 각 월, 공장에 대해 판매량, 생산량, 재고량 변수 초기화
for t in range(months):
    for j in range(factories):
        sold[(t, j)] = cp.Variable(productions, nonneg=True)
        manufacture[(t, j)] = cp.Variable(productions, nonneg=True)
        inventory[(t, j)] = cp.Variable(productions, nonneg=True)

# 목표 함수 정의
objective = cp.Maximize(
    # 판매로 인한 수익
    sum(cp.sum(cp.multiply(sold[(t, j)], price[j])) for t in range(months) for j in range(factories)) -
    # 재고 비용
    sum(cp.sum(cp.multiply(inventory[(t, j)], inventory_cost[t * factories + j])) for t in range(months) for j in range(factories)) -
    # 생산 비용
    sum(cp.sum(cp.multiply(manufacture[(t, j)], cost[t * factories + j])) for t in range(months) for j in range(factories)) #-
    # 광고 비용 (목표함수에서 제외)
    # sum(cp.sum(cp.multiply(manufacture[(t, j)], advertise_cost[j])) for t in range(months) for j in range(factories))
)

# 제약 조건
constraints = []

# 노동 시간 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], labor_time[j][:])) <= labor_limit[j])

# 포장 시간 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], packing_time[j][:])) <= packing_limit[j])

# 조립 시간 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][5:], assembly_time[j][5:])) <= assembly_limit[j])

# 생산량에 대한 광고비 제약
for t in range(months):
    c = 0
    for j in range(factories):
        c += cp.sum(cp.multiply(manufacture[(t, j)][:], advertise_cost[j][:]))
    constraints.append(c <= 20000)

# 생산 및 재고 균형 제약
for j in range(factories):
  for k in range(productions):
    # 첫 번째 달의 생산량은 첫 번째 달의 재고량과 첫 번째 달의 판매량의 합과 같아야 함
    constraints += [manufacture[(0, j)][k] == inventory[(0, j)][k] + sold[(0, j)][k]]

for j in range(factories):
  for k in range(productions):
    # 두 번째 달의 생산량과 이전 달의 재고량은 현재 달의 재고량과 판매량의 합과 같아야 함
    constraints += [manufacture[(1, j)][k] + inventory[(0, j)][k] == inventory[(1, j)][k] + sold[(1, j)][k]]

# 수요 제약
for i in range(months):
    for j in range(factories):
      for k in range(productions):
        # 각 제품의 판매량은 최소 수요 이상, 최대 수요 이하이어야 함
        constraints += [demand_bound[j][k] <= sold[(i, j)][k],
                        sold[(i, j)][k] <= demand_bound[factories + j][k]]

    # Graphite 생산 제약 조건
    constraints += [manufacture[(i, 0)][1] + manufacture[(i, 1)][1] + manufacture[(i, 2)][1] +
                    13 * (manufacture[(i, 0)][7] + manufacture[(i, 0)][8] +
                          manufacture[(i, 1)][7] + manufacture[(i, 1)][8] +
                          manufacture[(i, 2)][7] + manufacture[(i, 2)][8]) <= 4000]

# 문제 해결
prob = cp.Problem(objective, constraints)
prob.solve()

# 결과 출력
print(f"Optimal value: {prob.value}")

# DataFrame 생성
sold_data = []
manufacture_data = []
inventory_data = []

# 각 월, 공장의 판매량, 생산량, 재고량 결과 저장
for t in range(months):
    for j in range(factories):
        sold_data.append(sold[(t, j)].value)
        manufacture_data.append(manufacture[(t, j)].value)
        inventory_data.append(inventory[(t, j)].value)

# 제품명 리스트
name = ["Steel shafts", "Graphite shafts", "Forged iron heads", "Metal wood heads", "Titanium insert heads",
        "Set: Steel, metal", "Set: Steel, insert", "Set: Graphite, metal", "Set: Graphite, insert"]

# DataFrame 생성 및 인덱스 설정
sold_df = pd.DataFrame(sold_data, columns=name)
manufacture_df = pd.DataFrame(manufacture_data, columns=name)
inventory_df = pd.DataFrame(inventory_data, columns=name)

sold_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
manufacture_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
inventory_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

# 결과 출력
print("\nSold quantities:")
print(sold_df.transpose().round(2))
print("\nManufacture quantities:")
print(manufacture_df.transpose().round(2))
print("\nInventory quantities:")
print(inventory_df.transpose().round(2))

# Labor, Packing, , Advertising, Assembly, Graphite value 계산
labor = []
packing = []
assembly = []
advertise = []
graphite_usage = []

for t in range(months):
  for j in range(factories):
    labor.append(list(sold[(t, j)].value) @ labor_time[j][:])
    packing.append(list(sold[(t, j)].value) @ packing_time[j][:])
    advertise.append(list(sold[(t, j)].value) @ advertise_cost[j][:])
    assembly.append(list(sold[(t, j)].value) @ assembly_time[j][:])
    graphite_amount = manufacture[(t, j)][1].value + 13 * (manufacture[(t, j)][7].value + manufacture[(t, j)][8].value)
    graphite_usage.append(graphite_amount)

labor_df = pd.DataFrame(labor, columns=['Labor'])
packing_df = pd.DataFrame(packing, columns = ['Packing'])
advertise_df = pd.DataFrame(advertise, columns = ['Advertise'])
assembly_df = pd.DataFrame(assembly, columns = ['Assembly'])
graphite_df = pd.DataFrame(graphite_usage, columns=['Graphite Usage'])
labor_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
packing_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
advertise_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
assembly_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
graphite_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])


# 결과 출력
print("\nLabor value:")
print(labor_df.transpose().round(2))
print("\nPacking value:")
print(packing_df.transpose().round(2))
print("\nAdvertising Cost value:")
print(advertise_df.transpose().round(2))
print("\nAssembly time value:")
print(assembly_df.transpose().round(2))
print("\nGraphite usage value:")
print(graphite_df.transpose().round(2))

Optimal value: 258326.7623657477

Sold quantities:
Month                       1                         2                  
Factory                     1        2        3       1        2        3
Steel shafts              0.0     0.00     0.00     0.0     0.00     0.00
Graphite shafts         100.0   100.00  2000.00   100.0   100.00    50.00
Forged iron heads       200.0   200.00   100.00   200.0   200.00   100.00
Metal wood heads         30.0    30.00    15.00    30.0    30.00    15.00
Titanium insert heads  2000.0  2000.00  2000.00  2000.0  2000.00  2000.00
Set: Steel, metal         0.0     0.00     0.00     0.0     0.00     0.00
Set: Steel, insert        0.0     0.00     0.00     0.0     0.00     0.00
Set: Graphite, metal      0.0     0.00     0.00     0.0     0.00    83.57
Set: Graphite, insert    53.5    34.27    14.56    53.5    34.27     0.00

Manufacture quantities:
Month                       1                         2                  
Factory                     1       

In [ ]:
# 4(b) Labor Unlimited

import cvxpy as cp
import numpy as np
import pandas as pd

# 데이터 초기화
months = 2
factories = 3
productions = 9

# 가격, 비용, 재고 비용, 광고 비용, 노동 시간, 포장 시간, 조립 시간, 제약 조건 초기화
price = pd.read_excel('OR.xlsx', sheet_name = 'Revenue').values.transpose()

cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_2').values.transpose()), axis = 0)

inventory_cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_2').values.transpose()), axis = 0)

advertise_cost = pd.read_excel('OR.xlsx', sheet_name = 'Advertisement Cost').values.transpose()

labor_time = pd.read_excel('OR.xlsx', sheet_name = 'Labor').values.transpose()

labor_limit = np.array([12000, 15000, 22000])

packing_time = pd.read_excel('OR.xlsx', sheet_name = 'Packing').values.transpose()

packing_limit = np.array([20000, 40000, 35000])

assembly_time = pd.read_excel('OR.xlsx', sheet_name = 'Assembly_Time').values.transpose()

assembly_limit = np.array([5500, 5000, 6000])

demand_bound = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Demand Lower bound').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Demand Upper bound').values.transpose()), axis = 0)

for iter in range(factories):

  print(f"Labor Unlimited: Factory {iter + 1} / 3\n")

  # 변수 초기화
  sold = {}
  manufacture = {}
  inventory = {}

  for t in range(months):
      for j in range(factories):
          sold[(t, j)] = cp.Variable(productions, nonneg=True)
          manufacture[(t, j)] = cp.Variable(productions, nonneg=True)
          inventory[(t, j)] = cp.Variable(productions, nonneg=True)

  # 목표 함수
  objective = cp.Maximize(
      sum(cp.sum(cp.multiply(sold[(t, j)], price[j])) for t in range(months) for j in range(factories)) -
      sum(cp.sum(cp.multiply(inventory[(t, j)], inventory_cost[t * factories + j])) for t in range(months) for j in range(factories)) -
      sum(cp.sum(cp.multiply(manufacture[(t, j)], cost[t * factories + j])) for t in range(months) for j in range(factories)) #-
      #sum(cp.sum(cp.multiply(manufacture[(t, j)], advertise_cost[j])) for t in range(months) for j in range(factories))
  )

  # 제약 조건
  constraints = []

  # 노동 시간 제약
  for t in range(months):
      for j in range(factories):
          if j == iter:
            continue
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], labor_time[j][:])) <= labor_limit[j])

  # 포장 시간 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], packing_time[j][:])) <= packing_limit[j])

  # 조립 시간 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][5:], assembly_time[j][5:])) <= assembly_limit[j])

  # 생산량에 대한 광고비 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], advertise_cost[j][:])) <= 20000)

  # 생산 및 재고 균형 제약

  for j in range(factories):
    for k in range(productions):
      constraints += [manufacture[(0, j)][k] == inventory[(0, j)][k] + sold[(0, j)][k]]

  for j in range(factories):
    for k in range(productions):
      constraints += [manufacture[(1, j)][k] + inventory[(0, j)][k] == inventory[(1, j)][k] + sold[(1, j)][k]]

  # 수요 제약
  for i in range(months):
      for j in range(factories):
        for k in range(productions):
          constraints += [demand_bound[j][k] <= sold[(i, j)][k],
                          sold[(i, j)][k] <= demand_bound[factories + j][k]]
      constraints += [manufacture[(i, 0)][1] + manufacture[(i, 1)][1] + manufacture[(i, 2)][1] +
                      13 * (manufacture[(i, 0)][7] + manufacture[(i, 0)][8] +
                            manufacture[(i, 1)][7] + manufacture[(i, 1)][8] +
                            manufacture[(i, 2)][7] + manufacture[(i, 2)][8]) <= 4000]

  # 문제 해결
  prob = cp.Problem(objective, constraints)
  prob.solve()

  # 결과 출력
  print(f"Optimal value: {prob.value}")

  # DataFrame 생성
  sold_data = []
  manufacture_data = []
  inventory_data = []

  for t in range(months):
      for j in range(factories):
          sold_data.append(sold[(t, j)].value)
          manufacture_data.append(manufacture[(t, j)].value)
          inventory_data.append(inventory[(t, j)].value)

  name = ["Steel shafts", "Graphite shafts", "Forged iron heads", "Metal wood heads", "Titanium insert heads",
          "Set: Steel, metal", "Set: Steel, insert", "Set: Graphite, metal", "Set: Graphite, insert"]

  sold_df = pd.DataFrame(sold_data, columns=name)
  manufacture_df = pd.DataFrame(manufacture_data, columns=name)
  inventory_df = pd.DataFrame(inventory_data, columns=name)

  # Index 설정
  sold_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  manufacture_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  inventory_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

  # Labor, Packing, Assembly value 계산
  labor = []
  packing = []
  assembly = []
  for t in range(months):
    for j in range(factories):
      labor.append(list(sold[(t, j)].value) @ labor_time[j][:])
      packing.append(list(sold[(t, j)].value) @ packing_time[j][:])
      assembly.append(list(sold[(t, j)].value) @ assembly_time[j][:])
  labor_df = pd.DataFrame(labor, columns=['Labor'])
  packing_df = pd.DataFrame(packing, columns = ['Packing'])
  assembly_df = pd.DataFrame(assembly, columns = ['Assembly'])
  labor_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  packing_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  assembly_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

  # 결과 출력
  print("\nLabor value:")
  print(labor_df.transpose().round(2))
  print("\nPacking value:")
  print(packing_df.transpose().round(2))
  print("\nAssembly time value:")
  print(assembly_df.transpose().round(2))
  print("\nSold quantities:")
  print(sold_df.transpose().round(2))
  print("\nManufacture quantities:")
  print(manufacture_df.transpose().round(2))
  print("\nInventory quantities:")
  print(inventory_df.transpose().round(2))
  print("\n\n")

In [ ]:
# 4(b) Packing Unlimited

import cvxpy as cp
import numpy as np
import pandas as pd

# 데이터 초기화
months = 2
factories = 3
productions = 9

# 가격, 비용, 재고 비용, 광고 비용, 노동 시간, 포장 시간, 조립 시간, 제약 조건 초기화
price = pd.read_excel('OR.xlsx', sheet_name = 'Revenue').values.transpose()

cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_2').values.transpose()), axis = 0)

inventory_cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_2').values.transpose()), axis = 0)

advertise_cost = pd.read_excel('OR.xlsx', sheet_name = 'Advertisement Cost').values.transpose()

labor_time = pd.read_excel('OR.xlsx', sheet_name = 'Labor').values.transpose()

labor_limit = np.array([12000, 15000, 22000])

packing_time = pd.read_excel('OR.xlsx', sheet_name = 'Packing').values.transpose()

packing_limit = np.array([20000, 40000, 35000])

assembly_time = pd.read_excel('OR.xlsx', sheet_name = 'Assembly_Time').values.transpose()

assembly_limit = np.array([5500, 5000, 6000])

demand_bound = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Demand Lower bound').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Demand Upper bound').values.transpose()), axis = 0)

for iter in range(factories):

  print(f"Packing Unlimited: Factory {iter + 1} / 3\n")

  # 변수 초기화
  sold = {}
  manufacture = {}
  inventory = {}

  for t in range(months):
      for j in range(factories):
          sold[(t, j)] = cp.Variable(productions, nonneg=True)
          manufacture[(t, j)] = cp.Variable(productions, nonneg=True)
          inventory[(t, j)] = cp.Variable(productions, nonneg=True)

  # 목표 함수
  objective = cp.Maximize(
      sum(cp.sum(cp.multiply(sold[(t, j)], price[j])) for t in range(months) for j in range(factories)) -
      sum(cp.sum(cp.multiply(inventory[(t, j)], inventory_cost[t * factories + j])) for t in range(months) for j in range(factories)) -
      sum(cp.sum(cp.multiply(manufacture[(t, j)], cost[t * factories + j])) for t in range(months) for j in range(factories)) #-
      #sum(cp.sum(cp.multiply(manufacture[(t, j)], advertise_cost[j])) for t in range(months) for j in range(factories))
  )

  # 제약 조건
  constraints = []

  # 노동 시간 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], labor_time[j][:])) <= labor_limit[j])

  # 포장 시간 제약
  for t in range(months):
      for j in range(factories):
          if j == iter:
              continue
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], packing_time[j][:])) <= packing_limit[j])

  # 조립 시간 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][5:], assembly_time[j][5:])) <= assembly_limit[j])

  # 생산량에 대한 광고비 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], advertise_cost[j][:])) <= 20000)

  # 생산 및 재고 균형 제약

  for j in range(factories):
    for k in range(productions):
      constraints += [manufacture[(0, j)][k] == inventory[(0, j)][k] + sold[(0, j)][k]]

  for j in range(factories):
    for k in range(productions):
      constraints += [manufacture[(1, j)][k] + inventory[(0, j)][k] == inventory[(1, j)][k] + sold[(1, j)][k]]

  # 수요 제약
  for i in range(months):
      for j in range(factories):
        for k in range(productions):
          constraints += [demand_bound[j][k] <= sold[(i, j)][k],
                          sold[(i, j)][k] <= demand_bound[factories + j][k]]
      constraints += [manufacture[(i, 0)][1] + manufacture[(i, 1)][1] + manufacture[(i, 2)][1] +
                      13 * (manufacture[(i, 0)][7] + manufacture[(i, 0)][8] +
                            manufacture[(i, 1)][7] + manufacture[(i, 1)][8] +
                            manufacture[(i, 2)][7] + manufacture[(i, 2)][8]) <= 4000]

  # 문제 해결
  prob = cp.Problem(objective, constraints)
  prob.solve()

  # 결과 출력
  print(f"Optimal value: {prob.value}")

  # DataFrame 생성
  sold_data = []
  manufacture_data = []
  inventory_data = []

  for t in range(months):
      for j in range(factories):
          sold_data.append(sold[(t, j)].value)
          manufacture_data.append(manufacture[(t, j)].value)
          inventory_data.append(inventory[(t, j)].value)

  name = ["Steel shafts", "Graphite shafts", "Forged iron heads", "Metal wood heads", "Titanium insert heads",
          "Set: Steel, metal", "Set: Steel, insert", "Set: Graphite, metal", "Set: Graphite, insert"]

  sold_df = pd.DataFrame(sold_data, columns=name)
  manufacture_df = pd.DataFrame(manufacture_data, columns=name)
  inventory_df = pd.DataFrame(inventory_data, columns=name)

  # Index 설정
  sold_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  manufacture_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  inventory_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

  # Labor, Packing, Assembly value 계산
  labor = []
  packing = []
  assembly = []
  for t in range(months):
    for j in range(factories):
      labor.append(list(sold[(t, j)].value) @ labor_time[j][:])
      packing.append(list(sold[(t, j)].value) @ packing_time[j][:])
      assembly.append(list(sold[(t, j)].value) @ assembly_time[j][:])
  labor_df = pd.DataFrame(labor, columns=['Labor'])
  packing_df = pd.DataFrame(packing, columns = ['Packing'])
  assembly_df = pd.DataFrame(assembly, columns = ['Assembly'])
  labor_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  packing_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  assembly_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

  # 결과 출력
  print("\nLabor value:")
  print(labor_df.transpose().round(2))
  print("\nPacking value:")
  print(packing_df.transpose().round(2))
  print("\nAssembly time value:")
  print(assembly_df.transpose().round(2))
  print("\nSold quantities:")
  print(sold_df.transpose().round(2))
  print("\nManufacture quantities:")
  print(manufacture_df.transpose().round(2))
  print("\nInventory quantities:")
  print(inventory_df.transpose().round(2))
  print("\n\n")

In [ ]:
# 4(b) Assembly Time Unlimited

import cvxpy as cp
import numpy as np
import pandas as pd

# 데이터 초기화
months = 2
factories = 3
productions = 9

# 가격, 비용, 재고 비용, 광고 비용, 노동 시간, 포장 시간, 조립 시간, 제약 조건 초기화
price = pd.read_excel('OR.xlsx', sheet_name = 'Revenue').values.transpose()

cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_2').values.transpose()), axis = 0)

inventory_cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_2').values.transpose()), axis = 0)

advertise_cost = pd.read_excel('OR.xlsx', sheet_name = 'Advertisement Cost').values.transpose()

labor_time = pd.read_excel('OR.xlsx', sheet_name = 'Labor').values.transpose()

labor_limit = np.array([12000, 15000, 22000])

packing_time = pd.read_excel('OR.xlsx', sheet_name = 'Packing').values.transpose()

packing_limit = np.array([20000, 40000, 35000])

assembly_time = pd.read_excel('OR.xlsx', sheet_name = 'Assembly_Time').values.transpose()

assembly_limit = np.array([5500, 5000, 6000])

demand_bound = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Demand Lower bound').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Demand Upper bound').values.transpose()), axis = 0)

for iter in range(factories):

  print(f"Assembly Time Unlimited: Factory {iter + 1} / 3\n")

  # 변수 초기화
  sold = {}
  manufacture = {}
  inventory = {}

  for t in range(months):
      for j in range(factories):
          sold[(t, j)] = cp.Variable(productions, nonneg=True)
          manufacture[(t, j)] = cp.Variable(productions, nonneg=True)
          inventory[(t, j)] = cp.Variable(productions, nonneg=True)

  # 목표 함수
  objective = cp.Maximize(
      sum(cp.sum(cp.multiply(sold[(t, j)], price[j])) for t in range(months) for j in range(factories)) -
      sum(cp.sum(cp.multiply(inventory[(t, j)], inventory_cost[t * factories + j])) for t in range(months) for j in range(factories)) -
      sum(cp.sum(cp.multiply(manufacture[(t, j)], cost[t * factories + j])) for t in range(months) for j in range(factories)) #-
      #sum(cp.sum(cp.multiply(manufacture[(t, j)], advertise_cost[j])) for t in range(months) for j in range(factories))
  )

  # 제약 조건
  constraints = []

  # 노동 시간 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], labor_time[j][:])) <= labor_limit[j])

  # 포장 시간 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], packing_time[j][:])) <= packing_limit[j])

  # 조립 시간 제약
  for t in range(months):
      for j in range(factories):
          if j == iter:
              continue
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][5:], assembly_time[j][5:])) <= assembly_limit[j])

  # 생산량에 대한 광고비 제약
  for t in range(months):
      for j in range(factories):
          constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], advertise_cost[j][:])) <= 20000)

  # 생산 및 재고 균형 제약

  for j in range(factories):
    for k in range(productions):
      constraints += [manufacture[(0, j)][k] == inventory[(0, j)][k] + sold[(0, j)][k]]

  for j in range(factories):
    for k in range(productions):
      constraints += [manufacture[(1, j)][k] + inventory[(0, j)][k] == inventory[(1, j)][k] + sold[(1, j)][k]]

  # 수요 제약
  for i in range(months):
      for j in range(factories):
        for k in range(productions):
          constraints += [demand_bound[j][k] <= sold[(i, j)][k],
                          sold[(i, j)][k] <= demand_bound[factories + j][k]]
      constraints += [manufacture[(i, 0)][1] + manufacture[(i, 1)][1] + manufacture[(i, 2)][1] +
                      13 * (manufacture[(i, 0)][7] + manufacture[(i, 0)][8] +
                            manufacture[(i, 1)][7] + manufacture[(i, 1)][8] +
                            manufacture[(i, 2)][7] + manufacture[(i, 2)][8]) <= 4000]

  # 문제 해결
  prob = cp.Problem(objective, constraints)
  prob.solve()

  # 결과 출력
  print(f"Optimal value: {prob.value}")

  # DataFrame 생성
  sold_data = []
  manufacture_data = []
  inventory_data = []

  for t in range(months):
      for j in range(factories):
          sold_data.append(sold[(t, j)].value)
          manufacture_data.append(manufacture[(t, j)].value)
          inventory_data.append(inventory[(t, j)].value)

  name = ["Steel shafts", "Graphite shafts", "Forged iron heads", "Metal wood heads", "Titanium insert heads",
          "Set: Steel, metal", "Set: Steel, insert", "Set: Graphite, metal", "Set: Graphite, insert"]

  sold_df = pd.DataFrame(sold_data, columns=name)
  manufacture_df = pd.DataFrame(manufacture_data, columns=name)
  inventory_df = pd.DataFrame(inventory_data, columns=name)

  # Index 설정
  sold_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  manufacture_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  inventory_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

  # Labor, Packing, Assembly value 계산
  labor = []
  packing = []
  assembly = []
  for t in range(months):
    for j in range(factories):
      labor.append(list(sold[(t, j)].value) @ labor_time[j][:])
      packing.append(list(sold[(t, j)].value) @ packing_time[j][:])
      assembly.append(list(sold[(t, j)].value) @ assembly_time[j][:])
  labor_df = pd.DataFrame(labor, columns=['Labor'])
  packing_df = pd.DataFrame(packing, columns = ['Packing'])
  assembly_df = pd.DataFrame(assembly, columns = ['Assembly'])
  labor_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  packing_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
  assembly_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

  # 결과 출력
  print("\nLabor value:")
  print(labor_df.transpose().round(2))
  print("\nPacking value:")
  print(packing_df.transpose().round(2))
  print("\nAssembly time value:")
  print(assembly_df.transpose().round(2))
  print("\nSold quantities:")
  print(sold_df.transpose().round(2))
  print("\nManufacture quantities:")
  print(manufacture_df.transpose().round(2))
  print("\nInventory quantities:")
  print(inventory_df.transpose().round(2))
  print("\n\n")

In [ ]:
# Chandler - Packing, Glandale - Labor, Tucson - Packing Constraint Lifted Version
import cvxpy as cp
import numpy as np
import pandas as pd

# 데이터 초기화
months = 2
factories = 3
productions = 9

# 가격, 비용, 재고 비용, 광고 비용, 노동 시간, 포장 시간, 조립 시간, 제약 조건 초기화
price = pd.read_excel('OR.xlsx', sheet_name='Revenue').values.transpose()

cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name='Manufacturing Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name='Manufacturing Cost_2').values.transpose()), axis=0)

inventory_cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name='Inventory Cost_1').values.transpose(),
                                 pd.read_excel('OR.xlsx', sheet_name='Inventory Cost_2').values.transpose()), axis=0)

advertise_cost = pd.read_excel('OR.xlsx', sheet_name='Advertisement Cost').values.transpose()

labor_time = pd.read_excel('OR.xlsx', sheet_name='Labor').values.transpose()

labor_limit = np.array([12000, 15000, 22000])

packing_time = pd.read_excel('OR.xlsx', sheet_name='Packing').values.transpose()

packing_limit = np.array([20000, 40000, 35000])

assembly_time = pd.read_excel('OR.xlsx', sheet_name='Assembly_Time').values.transpose()

assembly_limit = np.array([5500, 5000, 6000])

demand_bound = np.concatenate((pd.read_excel('OR.xlsx', sheet_name='Demand Lower bound').values.transpose(),
                               pd.read_excel('OR.xlsx', sheet_name='Demand Upper bound').values.transpose()), axis=0)

# 변수 초기화
sold = {}
manufacture = {}
inventory = {}

for t in range(months):
    for j in range(factories):
        sold[(t, j)] = cp.Variable(productions, nonneg=True)
        manufacture[(t, j)] = cp.Variable(productions, nonneg=True)
        inventory[(t, j)] = cp.Variable(productions, nonneg=True)

# 목표 함수 정의
objective = cp.Maximize(
    sum(cp.sum(cp.multiply(sold[(t, j)], price[j])) for t in range(months) for j in range(factories)) -
    sum(cp.sum(cp.multiply(inventory[(t, j)], inventory_cost[t * factories + j])) for t in range(months) for j in range(factories)) -
    sum(cp.sum(cp.multiply(manufacture[(t, j)], cost[t * factories + j])) for t in range(months) for j in range(factories))
)

# 제약 조건
constraints = []

# 노동 시간 제약
for t in range(months):
    for j in range(factories):
        if j != 1:  # 공장 2 제외
            constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], labor_time[j][:])) <= labor_limit[j])

# 포장 시간 제약
for t in range(months):
    for j in range(factories):
        if j != 0 and j != 2:  # 공장 1 및 공장 3 제외
            constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], packing_time[j][:])) <= packing_limit[j])

# 조립 시간 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][5:], assembly_time[j][5:])) <= assembly_limit[j])

# 생산량에 대한 광고비 제약
    c = 0
    for j in range(factories):
        c += cp.sum(cp.multiply(manufacture[(t, j)][:], advertise_cost[j][:]))
    constraints.append(c <= 20000)

# 생산 및 재고 균형 제약
for j in range(factories):
    for k in range(productions):
        constraints += [manufacture[(0, j)][k] == inventory[(0, j)][k] + sold[(0, j)][k]]

for j in range(factories):
    for k in range(productions):
        constraints += [manufacture[(1, j)][k] + inventory[(0, j)][k] == inventory[(1, j)][k] + sold[(1, j)][k]]

# 수요 제약
for i in range(months):
    for j in range(factories):
        for k in range(productions):
            constraints += [demand_bound[j][k] <= sold[(i, j)][k],
                            sold[(i, j)][k] <= demand_bound[factories + j][k]]

# Graphite 생산 제약 조건
constraints += [manufacture[(i, 0)][1] + manufacture[(i, 1)][1] + manufacture[(i, 2)][1] +
                13 * (manufacture[(i, 0)][7] + manufacture[(i, 0)][8] +
                      manufacture[(i, 1)][7] + manufacture[(i, 1)][8] +
                      manufacture[(i, 2)][7] + manufacture[(i, 2)][8]) <= 4000 for i in range(months)]

# 문제 해결
prob = cp.Problem(objective, constraints)
prob.solve()

# 결과 출력
print(f"Optimal value: {prob.value}")

# DataFrame 생성
sold_data = []
manufacture_data = []
inventory_data = []

for t in range(months):
    for j in range(factories):
        sold_data.append(sold[(t, j)].value)
        manufacture_data.append(manufacture[(t, j)].value)
        inventory_data.append(inventory[(t, j)].value)

# 제품명 리스트
name = ["Steel shafts", "Graphite shafts", "Forged iron heads", "Metal wood heads", "Titanium insert heads",
        "Set: Steel, metal", "Set: Steel, insert", "Set: Graphite, metal", "Set: Graphite, insert"]

# DataFrame 생성 및 인덱스 설정
sold_df = pd.DataFrame(sold_data, columns=name)
manufacture_df = pd.DataFrame(manufacture_data, columns=name)
inventory_df = pd.DataFrame(inventory_data, columns=name)

sold_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
manufacture_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
inventory_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

# Labor, Packing, Assembly, Advertising value 계산
labor = []
packing = []
advertising = []
assembly = []
for t in range(months):
    for j in range(factories):
        labor.append(list(manufacture[(t, j)].value) @ labor_time[j][:])
        packing.append(list(manufacture[(t, j)].value) @ packing_time[j][:])
        advertising.append(list(manufacture[(t, j)].value) @ advertise_cost[j][:])
        assembly.append(list(manufacture[(t, j)].value) @ assembly_time[j][:])

labor_df = pd.DataFrame(labor, columns=['Labor'])
packing_df = pd.DataFrame(packing, columns=['Packing'])
advertising_df = pd.DataFrame(advertising, columns=['Advertising'])
assembly_df = pd.DataFrame(assembly, columns=['Assembly'])
labor_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
packing_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
advertising_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
assembly_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

# 결과 출력
print("\nSold quantities:")
print(sold_df.transpose().round(2))
print("\nManufacture quantities:")
print(manufacture_df.transpose().round(2))
print("\nInventory quantities:")
print(inventory_df.transpose().round(2))

# 각 공장, 월별로 사용한 Labor, Packing, Advertising 값 출력
print("\nLabor value:")
print(labor_df.transpose().round(2))
print("\nPacking value:")
print(packing_df.transpose().round(2))
print("\nAdvertising value:")
print(advertising_df.transpose().round(2))
print("\nAssembly time value:")
print(assembly_df.transpose().round(2))


In [10]:
# 4(c)

import cvxpy as cp
import numpy as np
import pandas as pd

# 데이터 초기화
months = 2
factories = 3
productions = 9

# 가격, 비용, 재고 비용, 광고 비용, 노동 시간, 포장 시간, 조립 시간, 제약 조건 초기화
price = pd.read_excel('OR.xlsx', sheet_name = 'Revenue').values.transpose()

cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Manufacturing Cost_2').values.transpose()), axis = 0)

inventory_cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Inventory Cost_2').values.transpose()), axis = 0)

advertise_cost = pd.read_excel('OR.xlsx', sheet_name = 'Advertisement Cost').values.transpose()

labor_time = pd.read_excel('OR.xlsx', sheet_name = 'Labor').values.transpose()

labor_limit = np.array([12000, 15000, 22000])

packing_time = pd.read_excel('OR.xlsx', sheet_name = 'Packing').values.transpose()

packing_limit = np.array([20000, 40000, 35000])

assembly_time = pd.read_excel('OR.xlsx', sheet_name = 'Assembly_Time').values.transpose()

assembly_limit = np.array([5500, 5000, 6000])

demand_bound = np.concatenate((pd.read_excel('OR.xlsx', sheet_name = 'Demand Lower bound').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name = 'Demand Upper bound').values.transpose()), axis = 0)

# 변수 초기화
sold = {}
manufacture = {}
inventory = {}

for t in range(months):
    for j in range(factories):
        sold[(t, j)] = cp.Variable(productions, nonneg=True)
        manufacture[(t, j)] = cp.Variable(productions, nonneg=True)
        inventory[(t, j)] = cp.Variable(productions, nonneg=True)

# 목표 함수
objective = cp.Maximize(
    sum(cp.sum(cp.multiply(sold[(t, j)], price[j])) for t in range(months) for j in range(factories)) -
    sum(cp.sum(cp.multiply(inventory[(t, j)], inventory_cost[t * factories + j])) for t in range(months) for j in range(factories)) -
    sum(cp.sum(cp.multiply(manufacture[(t, j)], cost[t * factories + j])) for t in range(months) for j in range(factories)) #-
    #sum(cp.sum(cp.multiply(manufacture[(t, j)], advertise_cost[j])) for t in range(months) for j in range(factories))
)

# 제약 조건
constraints = []

# 노동 시간 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], labor_time[j][:])) <= labor_limit[j])

# 포장 시간 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], packing_time[j][:])) <= packing_limit[j])

# 조립 시간 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][5:], assembly_time[j][5:])) <= assembly_limit[j])

# 생산량에 대한 광고비 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], advertise_cost[j][:])) <= 20000)

# 생산 및 재고 균형 제약

for j in range(factories):
  for k in range(productions):
    constraints += [manufacture[(0, j)][k] == inventory[(0, j)][k] + sold[(0, j)][k]]

for j in range(factories):
  for k in range(productions):
    constraints += [manufacture[(1, j)][k] + inventory[(0, j)][k] == inventory[(1, j)][k] + sold[(1, j)][k]]

# 수요 제약
for i in range(months):
    for j in range(factories):
      for k in range(productions):
        constraints += [demand_bound[j][k] <= sold[(i, j)][k],
                        sold[(i, j)][k] <= demand_bound[factories + j][k] * 1.5]

    constraints += [manufacture[(i, 0)][1] + manufacture[(i, 1)][1] + manufacture[(i, 2)][1] +
                    13 * (manufacture[(i, 0)][7] + manufacture[(i, 0)][8] +
                          manufacture[(i, 1)][7] + manufacture[(i, 1)][8] +
                          manufacture[(i, 2)][7] + manufacture[(i, 2)][8]) <= 4000]

# 문제 해결
prob = cp.Problem(objective, constraints)
prob.solve()

# 결과 출력
print(f"Optimal value: {prob.value}")

# DataFrame 생성
sold_data = []
manufacture_data = []
inventory_data = []

for t in range(months):
    for j in range(factories):
        sold_data.append(sold[(t, j)].value)
        manufacture_data.append(manufacture[(t, j)].value)
        inventory_data.append(inventory[(t, j)].value)

name = ["Steel shafts", "Graphite shafts", "Forged iron heads", "Metal wood heads", "Titanium insert heads",
        "Set: Steel, metal", "Set: Steel, insert", "Set: Graphite, metal", "Set: Graphite, insert"]

sold_df = pd.DataFrame(sold_data, columns=name)
manufacture_df = pd.DataFrame(manufacture_data, columns=name)
inventory_df = pd.DataFrame(inventory_data, columns=name)

# Index 설정
sold_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
manufacture_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
inventory_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

# Labor, Packing, Assembly value 계산
labor = []
packing = []
assembly = []
for t in range(months):
  for j in range(factories):
    labor.append(list(sold[(t, j)].value) @ labor_time[j][:])
    packing.append(list(sold[(t, j)].value) @ packing_time[j][:])
    assembly.append(list(sold[(t, j)].value) @ assembly_time[j][:])
labor_df = pd.DataFrame(labor, columns=['Labor'])
packing_df = pd.DataFrame(packing, columns = ['Packing'])
assembly_df = pd.DataFrame(assembly, columns = ['Assembly'])
labor_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
packing_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
assembly_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

# 결과 출력
print("\nLabor value:")
print(labor_df.transpose().round(2))
print("\nPacking value:")
print(packing_df.transpose().round(2))
print("\nAssembly time value:")
print(assembly_df.transpose().round(2))
print("\nSold quantities:")
print(sold_df.transpose().round(2))
print("\nManufacture quantities:")
print(manufacture_df.transpose().round(2))
print("\nInventory quantities:")
print(inventory_df.transpose().round(2))

Optimal value: 282606.47769676964

Labor value:
Month          1                           2                   
Factory        1        2         3        1        2         3
Labor    12000.0  15000.0  21637.67  12000.0  15000.0  21662.01

Packing value:
Month          1                          2                  
Factory        1        2        3        1        2        3
Packing  20000.0  21631.0  35000.0  20000.0  21631.0  35000.0

Assembly time value:
Month           1                  2              
Factory         1    2    3        1    2        3
Assembly  1591.04  0.0  0.0  1591.04  0.0  2967.36

Sold quantities:
Month                        1                         2                 
Factory                      1       2        3        1       2        3
Steel shafts              0.00     0.0     0.00     0.00     0.0     0.00
Graphite shafts         100.00   100.0  1334.33   100.00   100.0    50.00
Forged iron heads       200.00   200.0   100.00   200.00   200.0   10

In [8]:
# Additional Analysis 2 _ Goal Programming

import cvxpy as cp
import numpy as np
import pandas as pd

# 데이터 초기화
months = 2
factories = 3
productions = 9

# 가격, 비용, 재고 비용, 광고 비용, 노동 시간, 포장 시간, 조립 시간, 제약 조건 초기화
price = pd.read_excel('OR.xlsx', sheet_name='Revenue').values.transpose()

cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name='Manufacturing Cost_1').values.transpose(),
                       pd.read_excel('OR.xlsx', sheet_name='Manufacturing Cost_2').values.transpose()), axis=0)

inventory_cost = np.concatenate((pd.read_excel('OR.xlsx', sheet_name='Inventory Cost_1').values.transpose(),
                                 pd.read_excel('OR.xlsx', sheet_name='Inventory Cost_2').values.transpose()), axis=0)

advertise_cost = pd.read_excel('OR.xlsx', sheet_name='Advertisement Cost').values.transpose()

labor_time = pd.read_excel('OR.xlsx', sheet_name='Labor').values.transpose()

labor_limit = np.array([12000, 15000, 22000])

packing_time = pd.read_excel('OR.xlsx', sheet_name='Packing').values.transpose()

packing_limit = np.array([20000, 40000, 35000])

assembly_time = pd.read_excel('OR.xlsx', sheet_name='Assembly_Time').values.transpose()

assembly_limit = np.array([5500, 5000, 6000])

demand_bound = np.concatenate((pd.read_excel('OR.xlsx', sheet_name='Demand Lower bound').values.transpose(),
                               pd.read_excel('OR.xlsx', sheet_name='Demand Upper bound').values.transpose()), axis=0)

# 변수 초기화
sold = {}
manufacture = {}
inventory = {}

for t in range(months):
    for j in range(factories):
        sold[(t, j)] = cp.Variable(productions, nonneg=True)
        manufacture[(t, j)] = cp.Variable(productions, nonneg=True)
        inventory[(t, j)] = cp.Variable(productions, nonneg=True)

# 편차 변수
deviation_adv = cp.Variable(2, nonneg=True)

# 목표 함수 정의: 기존 목표 + 광고 비용 목표 편차 최소화
objective = cp.Maximize(
    # 판매로 인한 수익
    sum(cp.sum(cp.multiply(sold[(t, j)], price[j])) for t in range(months) for j in range(factories)) -
    # 재고 비용
    sum(cp.sum(cp.multiply(inventory[(t, j)], inventory_cost[t * factories + j])) for t in range(months) for j in range(factories)) -
    # 생산 비용
    sum(cp.sum(cp.multiply(manufacture[(t, j)], cost[t * factories + j])) for t in range(months) for j in range(factories)) -
    # 편차 최소화
    sum(deviation_adv)
)

# 제약 조건
constraints = []

# 노동 시간 제약
for t in range(months):
    for j in range(factories):
        if j != 1:  # 공장 2 제외
            constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], labor_time[j][:])) <= labor_limit[j])

# 포장 시간 제약
for t in range(months):
    for j in range(factories):
        if j != 0 and j != 2:  # 공장 1 및 공장 3 제외
            constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][:], packing_time[j][:])) <= packing_limit[j])

# 조립 시간 제약
for t in range(months):
    for j in range(factories):
        constraints.append(cp.sum(cp.multiply(manufacture[(t, j)][5:], assembly_time[j][5:])) <= assembly_limit[j])

# 생산 및 재고 균형 제약
for j in range(factories):
    for k in range(productions):
        constraints += [manufacture[(0, j)][k] == inventory[(0, j)][k] + sold[(0, j)][k]]

for j in range(factories):
    for k in range(productions):
        constraints += [manufacture[(1, j)][k] + inventory[(0, j)][k] == inventory[(1, j)][k] + sold[(1, j)][k]]

# 수요 제약
for i in range(months):
    for j in range(factories):
        for k in range(productions):
            constraints += [demand_bound[j][k] <= sold[(i, j)][k],
                            sold[(i, j)][k] <= demand_bound[factories + j][k]]

# Graphite 생산 제약 조건
constraints += [manufacture[(i, 0)][1] + manufacture[(i, 1)][1] + manufacture[(i, 2)][1] +
                13 * (manufacture[(i, 0)][7] + manufacture[(i, 0)][8] +
                      manufacture[(i, 1)][7] + manufacture[(i, 1)][8] +
                      manufacture[(i, 2)][7] + manufacture[(i, 2)][8]) <= 4000 for i in range(months)]

# 광고 비용 목표 제약 조건
advertising_cost_used = []
for t in range(months):
    total_adv_cost = sum(cp.sum(cp.multiply(manufacture[(t, j)], advertise_cost[j])) for j in range(factories))
    advertising_cost_used.append(total_adv_cost)
    # Goal 설정
    constraints.append(total_adv_cost + deviation_adv[t] == 23000)

# 문제 해결
prob = cp.Problem(objective, constraints)
prob.solve()

# 결과 출력
print(f"Optimal value: {prob.value}")

# DataFrame 생성
sold_data = []
manufacture_data = []
inventory_data = []

for t in range(months):
    for j in range(factories):
        sold_data.append(sold[(t, j)].value)
        manufacture_data.append(manufacture[(t, j)].value)
        inventory_data.append(inventory[(t, j)].value)

# 제품명 리스트
name = ["Steel shafts", "Graphite shafts", "Forged iron heads", "Metal wood heads", "Titanium insert heads",
        "Set: Steel, metal", "Set: Steel, insert", "Set: Graphite, metal", "Set: Graphite, insert"]

# DataFrame 생성 및 인덱스 설정
sold_df = pd.DataFrame(sold_data, columns=name)
manufacture_df = pd.DataFrame(manufacture_data, columns=name)
inventory_df = pd.DataFrame(inventory_data, columns=name)

sold_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
manufacture_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
inventory_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

# Labor, Packing, Assembly, Advertising value 계산
labor = []
packing = []
advertising = []
assembly = []
for t in range(months):
    for j in range(factories):
        labor.append(list(manufacture[(t, j)].value) @ labor_time[j][:])
        packing.append(list(manufacture[(t, j)].value) @ packing_time[j][:])
        advertising.append(list(manufacture[(t, j)].value) @ advertise_cost[j][:])
        assembly.append(list(manufacture[(t, j)].value) @ assembly_time[j][:])

labor_df = pd.DataFrame(labor, columns=['Labor'])
packing_df = pd.DataFrame(packing, columns=['Packing'])
advertising_df = pd.DataFrame(advertising, columns=['Advertising'])
assembly_df = pd.DataFrame(assembly, columns=['Assembly'])
labor_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
packing_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
advertising_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])
assembly_df.index = pd.MultiIndex.from_product([range(1, months+1), range(1, factories+1)], names=['Month', 'Factory'])

# 결과 출력
print("\nSold quantities:")
print(sold_df.transpose().round(2))
print("\nManufacture quantities:")
print(manufacture_df.transpose().round(2))
print("\nInventory quantities:")
print(inventory_df.transpose().round(2))

# 각 공장, 월별로 사용한 Labor, Packing, Advertising 값 출력
print("\nLabor value:")
print(labor_df.transpose().round(2))
print("\nPacking value:")
print(packing_df.transpose().round(2))
print("\nAdvertising value:")
print(advertising_df.transpose().round(2))
print("\nAssembly time value:")
print(assembly_df.transpose().round(2))

# 사용한 광고 비용 출력
for t in range(months):
    print(f"\nAdvertising cost used in month {t+1}: {advertising_cost_used[t].value}")


Optimal value: 308019.98910820275

Sold quantities:
Month                        1                          2                  
Factory                      1        2        3        1        2        3
Steel shafts            350.54     0.00     0.00  2000.00     0.00     0.00
Graphite shafts         100.00   118.37   398.30   100.00   118.37   398.30
Forged iron heads       200.00   200.00   100.00   200.00   200.00   100.00
Metal wood heads         30.00   492.89    15.00    30.00  1189.64    15.00
Titanium insert heads  1793.71  2000.00  2000.00  1381.35  2000.00  2000.00
Set: Steel, metal         0.00     0.00     0.00     0.00     0.00     0.00
Set: Steel, insert        0.00     0.00     0.00     0.00     0.00     0.00
Set: Graphite, metal      0.00     0.00    92.31     0.00     0.00    92.31
Set: Graphite, insert    84.62    83.33     0.00    84.62    83.33     0.00

Manufacture quantities:
Month                        1                          2                  
Factory    